In [ ]:
import numpy as np
import pandas as pd
import string
import nltk

nltk.download("stopwords")
nltk.download("punkt")
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /usr/local/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
DATASET_PATH = "./data/2cls_spam_text_cls.csv"
df = pd.read_csv(DATASET_PATH)

messages = df["Message"].values.tolist()
labels = df["Category"].values.tolist()

In [ ]:
def lowercase(text):
    return text.lower()


def punctuation_removal(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)


def tokenize(text):
    return nltk.word_tokenize(text)


def remove_stopwords(tokens):
    stop_words = nltk.corpus.stopwords.words("english")
    return [token for token in tokens if token not in stop_words]


def stemming(tokens):
    stemmer = nltk.PorterStemmer()
    return [stemmer.stem(token) for token in tokens]


def preprocess_text(text):
    text = lowercase(text)
    text = punctuation_removal(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = stemming(tokens)

    return tokens

In [16]:
messages = [preprocess_text(message) for message in messages]

print(messages)

[['go', 'jurong', 'point', 'crazi', 'avail', 'bugi', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'got', 'amor', 'wat'], ['ok', 'lar', 'joke', 'wif', 'u', 'oni'], ['free', 'entri', '2', 'wkli', 'comp', 'win', 'fa', 'cup', 'final', 'tkt', '21st', 'may', '2005', 'text', 'fa', '87121', 'receiv', 'entri', 'questionstd', 'txt', 'ratetc', 'appli', '08452810075over18'], ['u', 'dun', 'say', 'earli', 'hor', 'u', 'c', 'alreadi', 'say'], ['nah', 'dont', 'think', 'goe', 'usf', 'live', 'around', 'though'], ['freemsg', 'hey', 'darl', '3', 'week', 'word', 'back', 'id', 'like', 'fun', 'still', 'tb', 'ok', 'xxx', 'std', 'chg', 'send', '£150', 'rcv'], ['even', 'brother', 'like', 'speak', 'treat', 'like', 'aid', 'patent'], ['per', 'request', 'mell', 'mell', 'oru', 'minnaminungint', 'nurungu', 'vettam', 'set', 'callertun', 'caller', 'press', '9', 'copi', 'friend', 'callertun'], ['winner', 'valu', 'network', 'custom', 'select', 'receivea', '£900', 'prize', 'reward', 'claim', 'call', '09061701461', '

In [ ]:
def create_dictionary(messages):
    dictionary = []

    for tokens in messages:
        for token in tokens:
            if token not in dictionary:
                dictionary.append(token)

    return dictionary


def create_features(tokens, dictionary):
    features = np.zeros(len(dictionary))
    for token in tokens:
        if token in dictionary:
            features[dictionary.index(token)] += 1

    return features

In [20]:
dictionary = create_dictionary(messages)
X = np.array([create_features(tokens, dictionary) for tokens in messages])

print(dictionary)
print(X[0])

['go', 'jurong', 'point', 'crazi', 'avail', 'bugi', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'got', 'amor', 'wat', 'ok', 'lar', 'joke', 'wif', 'u', 'oni', 'free', 'entri', '2', 'wkli', 'comp', 'win', 'fa', 'cup', 'final', 'tkt', '21st', 'may', '2005', 'text', '87121', 'receiv', 'questionstd', 'txt', 'ratetc', 'appli', '08452810075over18', 'dun', 'say', 'earli', 'hor', 'c', 'alreadi', 'nah', 'dont', 'think', 'goe', 'usf', 'live', 'around', 'though', 'freemsg', 'hey', 'darl', '3', 'week', 'word', 'back', 'id', 'like', 'fun', 'still', 'tb', 'xxx', 'std', 'chg', 'send', '£150', 'rcv', 'even', 'brother', 'speak', 'treat', 'aid', 'patent', 'per', 'request', 'mell', 'oru', 'minnaminungint', 'nurungu', 'vettam', 'set', 'callertun', 'caller', 'press', '9', 'copi', 'friend', 'winner', 'valu', 'network', 'custom', 'select', 'receivea', '£900', 'prize', 'reward', 'claim', 'call', '09061701461', 'code', 'kl341', 'valid', '12', 'hour', 'mobil', '11', 'month', 'r', 'entitl', 'updat', 'late

In [24]:
print(set(labels))

{'spam', 'ham'}


In [21]:
le = LabelEncoder()
y = le.fit_transform(labels)

In [25]:
VAL_SIZE = 0.2
TEST_SIZE = 0.125
SEED = 0

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=VAL_SIZE, shuffle=True, random_state=SEED
)

X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, test_size=TEST_SIZE, shuffle=True, random_state=SEED
)

In [26]:
%%time
model = GaussianNB()
print('Start training .....')
model = model.fit(X_train, y_train)
print('Training completed')

Start training .....
Training completed
CPU times: user 456 ms, sys: 330 ms, total: 785 ms
Wall time: 864 ms


In [30]:
def predict(text, model, dictionary):
    processed_text = preprocess_text(text)
    features = create_features(processed_text, dictionary)
    features = np.array(features).reshape(1, -1)
    prediction = model.predict(features)
    prediction_cls = le.inverse_transform(prediction)[0]

    return prediction_cls

In [31]:
test_input = 'I am actually thinking a way of doing something useful'
prediction_cls = predict(test_input, model, dictionary)
print(f'Predictions: {prediction_cls}')

Predictions: ham


In [28]:
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

val_accuracy = accuracy_score(y_val, y_val_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f'Val accuracy: {val_accuracy}')
print(f'Test accuracy: {test_accuracy}')

Val accuracy: 0.8816143497757848
Test accuracy: 0.8602150537634409
